In [3]:
#
# restrictor - un programme simple pour rechercher des sites de
#              restriction dans un fichier d'ADN en format FASTA
#
# IN: 
#  - Un fichier de séquence d'ADN en format FASTA
#
# OUT:
#  - Une sortie écran du déroulement du programme
#  - Un fichier en format texte avec les résultats 
#    sous forme tabulée
#
#
# Modules nécessaires
#
import os
import re
import sys

#
# Constantes utiles pour le travail à venir...
# Il y a une raison pourquoi j'ai mis des parenthèses carrées :-)
#
# On utilise les codes IUPAC:
#
iupac = {
  "A" : "A",
  "C" : "C",
  "G" : "G",
  "T" : "T",
  "R" : "[AG]",
  "Y" : "[CT]",
  "S" : "[GC]",
  "W" : "[AT]",
  "K" : "[GT]",
  "M" : "[AC]",
  "B" : "[CGT]",
  "D" : "[AGT]",
  "H" : "[ACT]",
  "V" : "[ACG]",
  "N" : "[ACGT]",
  "." : ".", 
  "-" : "-"
}
#
# Acquisition du fichier de séquences avec PATH complet
#
aSeq = input("Où se trouve le fichier (avec le PATH complet)?")
#
# Verification que le fichier existe
#
if not (os.path.exists(aSeq)):
    print("Le fichier nomme "+aSeq+" n'existe pas...")
    sys.exit()
else:
    print("Fichier trouve! Lecture...")
#
# Lecture du fichier
#
aFile = open(aSeq,"r")
allLines = aFile.readlines()
# Entete du fichier
header = ""
#
# Verifions que le fichier a le bon format sans utiliser Biopython:
# La première ligne de tout fichier FASTA doit commencer par ">"
#
print("Verification conformite au format FASTA")
if not allLines[0][0] == '>':
    print("Erreur!! La sequence ne commence pas par \">\" donc n'est pas FASTA...")
    sys.exit()
else:
    header = allLines[0]
    print("Fichier conforme!!")
#    
# On retire la ligne de commentaires, maintenant stocké dans la variable header
#
allLines.pop(0)
daSeq = ""
for aLine in allLines:
    # Il faut retirer les retours de chariot
    aLine = aLine.replace("\n","")
    # On veut concatener les lignes de sequence pour n'avoir qu'une seule
    # ligne en mémoire
    daSeq = daSeq+aLine
#
# Verifions que le fichier contient bel et bien une sequence d'ADN
#
# Comment faire? Recherchons un caractère qui n'existe pas dans 
# l'alphabet de l'ADN accepté par IUPAC: on recherche un caractere de la sequence 
# qui n'est pas une clé du dictionnaire iupac...
#
print("Verification que la sequence est une sequence d'ADN...")
for aNuc in daSeq:
    if not aNuc in iupac:
        print("Erreur!! La sequence n'est pas une sequence de nucléotides mais d'acides amines...")
        sys.exit()
else:
    print("Le fichier contient des nucléotides!!")
    
#
# Ok, on sait que notre fichier existe, a le bon format et contient des nucléotides
# De plus, nous avons préparé deux variables: la ligne de commentaires et une
# contenant une longue chaine de caractères (la séquence) pour la recherche des motifs
#
# On doit maintenant construire une structure de données à partir du fichier link_strider.txt
# pour y mettre les enzymes et les infos associées
#
# Détails importants:
#
# - On doit retirer l'en-tete du fichier qui ne nous dit pas grand chose, 
#   à part la ligne 9 qui donne la version du fichier.
# - On ne veut que les enzymes de Type II: le site de coupure est inclus dans le motif recherché
#   Dans le fichier, ce sont les enzymes dont le premier caractère N'EST PAS un "#"!
# - Les motifs du fichier sont en lettres minuscules alors que notre sequence est écrite en lettres
#   majuscules.
# - Il faudra tenir compte du fait que certains enzymes ont des sites 
#   dégénérés: par ex., le site ApoI, r/aatty veut dire en fait (AG)/AATT(CT) 
#

#
# On va assumer que le fichier link_strider.txt existe :-)
#
# Pour une installation Jupyter locale
rezFile = open("../z.misc_files/data_misc/link_strider.txt")
allData = rezFile.readlines()
#
# On va retirer les 8 premières lignes
#
for i in range(8):
    allData.pop(0)
    i += 1
#
# On intercepte la ligne 9 et on retire la ligne 10
#
version = allData.pop(0)
version = version.split(" ")
# Question: pourquoi est-je besoin de la valeur de len(version) pour la suite?
# Variable à utiliser plus tard dans le rapport
version = version[53]+"/"+version[55]
allData.pop(0)
#
# On lit maintenant le fichier des enzymes une ligne à la fois en créant une
# structure de données unique contenant toute les infos nécessaires pour le rapport
# voulu
#
# - Chaque ligne possède deux champs pour un enzyme Type II
#     - le nom;
#     - le site avec la position de la coupure
#
# Liste de tous les enzymes
#
allOurEnzymes = list()
k = 0
for aLine in allData:
    if not '#' in aLine:
        tmpData = aLine.split(",")
        tmpName = tmpData[0]
        tmpSite = tmpData[1]
        tmpPos = tmpSite.rfind("/")
        tmpFinder = tmpSite.replace("/","")
        tmpRE = ""
        #
        # Il faut transformer le site en expression réguliére conforme
        # C'est pour ça que le dictionnaire iupac est utile
        # On remplace simplement le caractère ambigue par le patron correspondant
        #
        for aNuc in tmpSite:
             if aNuc.upper() in iupac:
                tmpRE = tmpRE+iupac[aNuc.upper()]      
        # Construction du compilateur de recherche
        eng = re.compile(tmpRE,re.IGNORECASE)
        # Dictionaire pour chaque enzyme        
        anEnzyme = {
            "Nom" : tmpName,
            "Site": tmpSite,
            "SiteRE": tmpRE,
            "Offsite" : tmpPos,
            "RE": eng
        }
        allOurEnzymes.append(anEnzyme)
#
# Ok maintenant on a tous les outils pour travailler
#  - Notre info sur la séquence dans la variable header
#  - Notre sequence dans la variable daSeq
#  - Toutes les infos disponibles sur les enzyme de restrictions dans la liste
#    allOurEnzymes sous la forme de dictionnaires pour chaque enzyme
#
output = input("Donner un nom au fichier de sortie, incluant la PATH: ")
#
# Écriture du fichier de résultat
#
results = open(output+".restrict","w")
head = "Restrictor - Un programme simplet pour recherche des sites de \nrestriction d'une sequence FASTA"
results.write(head+"\n")
results.write("=> Infos sur la sequence:\n")
results.write(header+"\n")
results.write("=> Version de la base de donnes REBASE:\n")
results.write(version+"\n")
results.write("<==================== RESULTATS =====================>\n\n")
for z in allOurEnzymes:
    results.write("\nNOM\t\t\tSITE\t\t\t\tMOTIF RECHERCHE\n")
    results.write(z["Nom"]+ "\t\t" +z["Site"]+ "\t\t\t" +z["SiteRE"]+"\n")
    results.write("\n== Positions des coupures pour {} ==\n\n".format(z["Nom"]))
    # Retourne un iterateur avec les résultats 
    allSites = z["RE"].finditer(daSeq)
    # Retourne une liste, vide si pas de sites trouvés
    iterable = re.findall(z["SiteRE"],daSeq,re.IGNORECASE)
    # Verifier si l'enzyme à digérer quelque chose...    
    if len(iterable) > 0:
      for a in allSites:
        pos = a.span()[0] + int(z["Offsite"])
        results.write(a.group()+" : "+str(pos)+"\n")
    else:
        results.write("==> Pas de sites pour {}\n".format(z["Nom"]))
    results.write("\n------------\n")
print("Done!!")
results.close()

Où se trouve le fichier (avec le PATH complet)? /Volumes/foisys/Documents/Cours_UdeM_CVM-Contenu/UdeM/BCM3553_Bio-informatique_pr_BCM/Jupyter-bcm3553/z.misc_files/data_seq/h.sapiens.cttn.iso.a.cdna.fa


Fichier trouve! Lecture...
Verification conformite au format FASTA
Fichier conforme!!
Verification que la sequence est une sequence d'ADN...
Le fichier contient des nucléotides!!


Donner un nom au fichier de sortie, incluant la PATH:  /Volumes/foisys/Documents/Cours_UdeM_CVM-Contenu/UdeM/BCM3553_Bio-informatique_pr_BCM/Jupyter-bcm3553/test.restrictor


Done!!
